In [23]:
import cv2
import face_recognition

# Imagen a comparar

In [24]:
image = cv2.imread("Imagen/Jhonnatan1.jpg")
face_loc = face_recognition.face_locations(image)[0]
#print("face_loc:", face_loc)
face_image_encodings = face_recognition.face_encodings(image, known_face_locations=[face_loc])[0]
#print("face_image_encodings:", face_image_encodings)

In [25]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml') 
#eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml') 
smile_cascade = cv2.CascadeClassifier('haarcascade_smile.xml') 

In [26]:
#cv2.rectangle(image, (face_loc[3], face_loc[0]), (face_loc[1], face_loc[2]), (0,255,0))
#cv2.imshow("Image", image)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

In [27]:
def detect(gray, frame):
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
        #cv2.rectangle(frame,(x, y),((x + w),(y + h)),(255, 0, 0), 2)
        roi_gray = gray[y:y + h, x:x + w]
        roi_color = frame[y:y + h, x:x + w]
        smiles = smile_cascade.detectMultiScale(roi_gray, 1.8, 20)
  
        for (sx, sy, sw, sh) in smiles:
            cv2.rectangle(roi_color,(sx, sy),((sx + sw),(sy + sh)),(0, 0, 255), 2)
    return frame

# Video Streaming

In [29]:
cap =  cv2.VideoCapture(0, cv2.CAP_DSHOW)

while True:
    ret, frame = cap.read()
    if ret == False:
        break
    frame = cv2.flip(frame, 1)
    
    face_locations = face_recognition.face_locations(frame)
    
    if face_locations != []:
        for face_location in face_locations:
            
            face_frame_encodings = face_recognition.face_encodings(frame, known_face_locations = [face_location])[0]
            result = face_recognition.compare_faces([face_frame_encodings], face_image_encodings)
            #print("Result:", result)
            
            if result[0] == True:
                text = "Jhonnatan"
                color = (123,228,0)
            else:
                text = "Desconocido"
                color = (50,50,255)
            
            cv2.rectangle(frame, (face_location[3], face_location[2]), (face_location[1], face_location[2]+30 ), color, -1)
            cv2.rectangle(frame, (face_location[3], face_location[0]), (face_location[1], face_location[2]), color, 2)
            cv2.putText(frame, text, (face_location[3], face_location[2] +20 ), 2, 0.7, (255, 255, 255), 1 )
            
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)           
            canvas = detect(gray, frame)    
    
    
    cv2.imshow("Video", frame)
    k = cv2.waitKey(1)
    
    if k == 27 & 0xFF:
        break

cap.release()
cv2.destroyAllWindows()